In [2]:
import pandas as pd
import numpy as np
import ast

pd.set_option("display.max_rows", None)

In [3]:
df = pd.read_csv(r"../output/kfold.csv")

In [4]:
# List of metric names
metric_names = [
    "f1",
    "mse",
    "accuracy",
    "recall",
    "precision",
    "auc",
    "area_under_pr",
    "r2_score",
    "rmse",
]
dataset = df.copy()
# Create columns based on metric names
# Convert string representations to dictionaries
dataset["output_metrics"] = dataset["output_metrics"].apply(ast.literal_eval)
# Create columns based on metric names
for metric in metric_names:
    dataset[metric] = dataset["output_metrics"].apply(
        lambda x: x.get(metric) if isinstance(x, dict) else np.nan
    )

In [5]:
df

,run_id,run_config,dataset,model,execution_mode,eval_metric,best_params,best_score,score_std,output_metrics,saved_model_path,run_time
0,202309-1713-2319-2541b093-306a-41cc-9e60-6b2fc...,"{'dataset': 'iris', 'model': 'xgb', 'best_para...",iris,xgb,hyperopt_kfold,accuracy,"{'alpha': 3.9316627966597033, 'colsample_bytre...",0.946667,0.026667,"{'accuracy': [0.9666666666666667, 0.9666666666...",./output/modelsaves/iris/xgb/202309-1713-2319-...,2.860786
1,202309-1713-2321-50e57c58-0154-4c76-8c5a-d497e...,"{'dataset': 'iris', 'model': 'catboost', 'best...",iris,catboost,hyperopt_kfold,accuracy,"{'bagging_temperature': 0.14228166126511824, '...",0.960000,0.038873,"{'accuracy': [1.0, 0.9666666666666667, 0.93333...",./output/modelsaves/iris/catboost/202309-1713-...,34.557710
2,202309-1713-2356-98436ca0-49db-4b1c-923b-b8f99...,"{'dataset': 'titanic', 'model': 'xgb', 'best_p...",titanic,xgb,hyperopt_kfold,accuracy,"{'alpha': 3.3470534018722558, 'colsample_bytre...",0.832754,0.028829,"{'recall': [0.7246376811594203, 0.764705882352...",./output/modelsaves/titanic/xgb/202309-1713-23...,6.943799
3,202309-1713-2403-62146a4d-9ac6-40d5-8d5e-32bcc...,"{'dataset': 'titanic', 'model': 'catboost', 'b...",titanic,catboost,hyperopt_kfold,accuracy,"{'bagging_temperature': 0.9928196516799518, 'd...",0.830513,0.028147,"{'recall': [0.7246376811594203, 0.764705882352...",./output/modelsaves/titanic/catboost/202309-17...,64.763489


In [4]:
# Specify the metric you want to maximize
metric_to_maximize = "accuracy"
tomax = {
    "f1": True,
    "mse": False,
    "accuracy": True,
    "recall": True,
    "precision": True,
    "auc": True,
    "area_under_pr": True,
    "rmse": False,
    "r2_score": True,
}


# Filter out rows with NaN values in the specified metric column
filtered_df = dataset.dropna(subset=[metric_to_maximize]).loc[
    dataset[metric_to_maximize] != "nan"
]

# Find the rows that maximize the specified metric for each dataset
if tomax[metric_to_maximize]:
    max_rows = filtered_df.sort_values(["dataset", metric_to_maximize], ascending=False)
else:
    max_rows = filtered_df.sort_values(["dataset", metric_to_maximize], ascending=True)

sel_cols = [
    "dataset",
    "model",
    "run_time",
    "eval_metric",
    "best_params",
    "output_metrics",
    "debug_preds",
    "f1",
    "mse",
    "accuracy",
    "recall",
    "precision",
    "auc",
    "area_under_pr",
    "r2_score",
    "rmse",
]

dataset_name = "heloc"
max_rows[sel_cols].head(50)

,dataset,model,run_time,eval_metric,best_params,output_metrics,debug_preds,f1,mse,accuracy,recall,precision,auc,area_under_pr,r2_score,rmse
47,heloc,catboost,278.405546,roc_auc,"{'bagging_temperature': 0.9112047589768257, 'd...","{'recall': 0.7730287398673544, 'precision': 0....","[1, 1, 1, 1, 1, 0, 1, 0, 0, 1]",0.740296,NaN,0.718547,0.773029,0.710223,0.793982,0.666805,NaN,NaN
48,heloc,xgb,1.401828,roc_auc,"{'alpha': 3.367587079252083, 'colsample_bytree...","{'recall': 0.7745025792188651, 'precision': 0....","[1, 1, 1, 1, 1, 0, 1, 0, 0, 1]",0.740662,NaN,0.718547,0.774503,0.709656,0.789792,0.666647,NaN,NaN
56,heloc,fttransformer,296.456918,roc_auc,"{'AdamW_weight_decay': 3.059778932803039e-05, ...","{'recall': 0.7627118644067796, 'precision': 0....","[1, 1, 1, 0, 1, 0, 0, 0, 0, 0]",0.736393,NaN,0.716635,0.762712,0.711829,0.787427,0.666057,NaN,NaN
57,heloc,categoryembedding,59.775566,roc_auc,"{'AdamW_weight_decay': 5.715467320011227e-05, ...","{'recall': 0.7715549005158437, 'precision': 0....","[1, 1, 1, 0, 1, 0, 1, 0, 0, 1]",0.738364,NaN,0.716252,0.771555,0.707911,0.784286,0.664739,NaN,NaN
59,heloc,node,134.680852,roc_auc,"{'AdamW_weight_decay': 4.169132014394808e-05, ...","{'recall': 0.7745025792188651, 'precision': 0....","[1, 1, 1, 0, 1, 0, 1, 0, 0, 1]",0.738321,NaN,0.715105,0.774503,0.705369,0.785590,0.663327,NaN,NaN
58,heloc,gandalf,78.293185,roc_auc,"{'AdamW_weight_decay': 6.114100305791925e-05, ...","{'recall': 0.7487103905674282, 'precision': 0....","[1, 0, 1, 0, 1, 0, 1, 0, 0, 0]",0.730935,NaN,0.713958,0.748710,0.713985,0.783450,0.664969,NaN,NaN
55,heloc,gate,967.443203,roc_auc,"{'AdamW_weight_decay': 3.654135403850366e-05, ...","{'recall': 0.735445836403832, 'precision': 0.7...","[1, 0, 1, 0, 1, 0, 1, 0, 0, 0]",0.727140,NaN,0.713576,0.735446,0.719020,0.778653,0.666085,NaN,NaN
49,heloc,mlp,458.054062,roc_auc,"{'solver': 'adam', 'learning_rate_init': 0.01,...","{'recall': 0.7435519528371407, 'precision': 0....","[1, 1, 1, 0, 1, 0, 1, 0, 0, 0]",0.727469,NaN,0.710899,0.743552,0.712068,0.775995,0.662538,NaN,NaN
61,heloc,tabtransformer,53.846923,roc_auc,"{'AdamW_weight_decay': 1.9498395973771847e-05,...","{'recall': 0.6927044952100221, 'precision': 0....","[1, 1, 1, 1, 1, 0, 0, 0, 0, 1]",0.706236,NaN,0.700956,0.692704,0.720307,0.771520,0.658424,NaN,NaN
54,heloc,tabnet,155.900223,roc_auc,"{'AdamW_weight_decay': 6.211024420297632e-05, ...","{'recall': 0.6919675755342668, 'precision': 0....","[1, 0, 1, 1, 1, 0, 0, 0, 0, 1]",0.702581,NaN,0.695985,0.691968,0.713526,0.763490,0.653584,NaN,NaN


In [ ]:
max_rows.loc[max_rows["model"] == "fttransformer"]["best_params"].iloc[0]

In [ ]:
d = {
    "AdamW_weight_decay": 0.009146661940655405,
    "Adam_weight_decay": 0.0026893683659541257,
    "ExponentialLR_gamma": 0.9813824711975316,
    "ReduceLROnPlateau_factor": 0.11738557751606361,
    "ReduceLROnPlateau_patience": 6,
    "StepLR_gamma": 0.9851456952449655,
    "StepLR_step_size": 29,
    "add_norm_dropout": 0.0992106504569584,
    "attn_dropout": 0.20163724049581883,
    "batch_size": 2496,
    "embedding_dropout": 0.056162111868724864,
    "embedding_initialization": "kaiming_uniform",
    "ff_dropout": 0.09808690887243214,
    "ff_hidden_multiplier": 4,
    "learning_rate": 0.0007179902919953625,
    "num_attn_blocks": 6,
    "shared_embedding_fraction": 0.4529901637185145,
    "transformer_activation": "ReGLU",
    "outer_params": {
        "hyperopt_evals": 10,
        "auto_lr_find": False,
        "max_epochs": 1000,
        "val_size": 0.15,
        "early_stopping_patience": 6,
    },
}

In [ ]:
d

In [ ]:
maximized_rows["best_params"].iloc[1]